<a href="https://colab.research.google.com/github/DkCodeProjct/cs50_AI_ML/blob/main/trafficFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

print(tf.__version__)

2.17.0


In [2]:
!unzip /content/gtsrb.zip -d /content/traffic/gtsrb


Archive:  /content/gtsrb.zip
checkdir:  cannot create extraction directory: /content/traffic/gtsrb
           No such file or directory


In [8]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [9]:

# Create the necessary directories
os.makedirs('/content/traffic/gtsrb', exist_ok=True)

# Step 2: Unzip the file into the directory
!unzip /content/gtsrb.zip -d /content/traffic/gtsrb

Archive:  /content/gtsrb.zip
replace /content/traffic/gtsrb/gtsrb/0/00004_00023.ppm? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:

main_path = '/content/traffic/gtsrb/gtsrb'

directories = [d for d in os.listdir(main_path) if os.path.isdir(os.path.join(main_path, d))]

# Print the number of directories
print(f"Number of directories: {len(directories)}")

Number of directories: 6


In [10]:

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 6
TEST_SIZE = 0.2


In [6]:

def load_data(data_dir):
    images = []
    labels = []

    # Loop over all categories
    for category in range(NUM_CATEGORIES):
        cat_path = os.path.join(data_dir, str(category))
        for filename in os.listdir(cat_path):
            img_path = os.path.join(cat_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            images.append(img)
            labels.append(category)

    return images, labels

In [11]:
images, labels = load_data('/content/traffic/gtsrb/gtsrb')

# Convert images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    images, labels, test_size=TEST_SIZE, random_state=42
)

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CATEGORIES)

# Print the shape of the datasets
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (5352, 30, 30, 3)
y_train shape: (5352, 6)
x_test shape: (1338, 30, 30, 3)
y_test shape: (1338, 6)


In [12]:
# Train The model../

def get_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CATEGORIES, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Get the model
model = get_model()

# Print a summary of the model
model.summary()

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 12, 12, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 158,502 (619.15 KB)

 Trainable params: 158,502 (619.15 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.2690 - loss: 10.0697 - val_accuracy: 0.4574 - val_loss: 1.4328
Epoch 2/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.4647 - loss: 1.3757 - val_accuracy: 0.6839 - val_loss: 0.9597
Epoch 3/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.6485 - loss: 0.9665 - val_accuracy: 0.8184 - val_loss: 0.5920
Epoch 4/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.7457 - loss: 0.7219 - val_accuracy: 0.8498 - val_loss: 0.5089
Epoch 5/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8165 - loss: 0.5617 - val_accuracy: 0.8901 - val_loss: 0.3657
Epoch 6/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8493 - loss: 0.4499 - val_accuracy: 0.8871 - val_loss: 0.3697
Epoch 7/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.8785 - loss: 0.3631 - val_accuracy: 0.9051 - val_loss: 0.3088
Epoch 8/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - accuracy: 0.8832 - loss: 0.3514 - val_

In [13]:
# Save the model to a file
model.save('gtrsbWellTRAINED.h5')

print("Model saved to my_gtsrb_model.h5")


Model saved to my_gtsrb_model.h5


In [14]:
model.save('/content/gtrsbWellTRAINED.h5')
print("Model saved to /content/my_gtsrb_model.h5")


Model saved to /content/my_gtsrb_model.h5


In [15]:
import os

# List files in the /content directory
print(os.listdir('/content'))


['.config', 'gtsrb.zip', 'traffic', 'gtrsbWellTRAINED.h5', 'sample_data']
